# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [5]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from tqdm.notebook import tqdm
import joblib
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import ParameterGrid
from sklearn.base import clone
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib

C:\ProgramData\Anaconda3\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [9]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame.")
        
        X['hour'] = pd.to_datetime(X['timestamp']).dt.hour
        X['dayofweek'] = pd.to_datetime(X['timestamp']).dt.weekday
        
        # Drop the original timestamp column
        X = X.drop(columns=['timestamp'])
        
        return X

In [11]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target_column):
        self.target_column = target_column
        self.encoder = OneHotEncoder(sparse_output=False, drop='first')

    def fit(self, X, y=None):
        # Определите категориальные признаки, кроме целевого
        self.categorical_features_ = X.select_dtypes(include=['object', 'category']).columns.tolist()
        if self.target_column in self.categorical_features_:
            self.categorical_features_.remove(self.target_column)

        # Поддержка только категориальных признаков
        self.encoder.fit(X[self.categorical_features_])
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame.")

        y = X[self.target_column]
        X_features = X.drop(columns=[self.target_column])

        # Примените OneHotEncoder к категориальным признакам
        X_encoded = self.encoder.transform(X_features[self.categorical_features_])
        encoded_columns = self.encoder.get_feature_names_out(self.categorical_features_)

        # Создайте DataFrame из закодированных признаков
        X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_columns, index=X.index)

        # Объедините закодированные признаки с оставшимися столбцами
        X_non_categorical = X_features.drop(columns=self.categorical_features_)
        X_result = pd.concat([X_non_categorical, X_encoded_df, y], axis=1)

        return X_result

In [13]:
class TrainValidationTest:
    def __init__(self, test_size=0.2, random_state=21):
        self.test_size = test_size
        self.random_state = random_state
    
    def split(self, X, y):
        # First split X and y into training + validation, and testing
        X_train_val, X_test, y_train_val, y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, stratify=y
        )
        
        # Then split training + validation into training and validation
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_train_val, y_train_val, test_size=self.test_size, random_state=self.random_state, stratify=y_train_val
        )
        
        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [17]:
def fit_model(params, estimator, X_train, y_train):
        # Клонируем модель и устанавливаем параметры
        model = clone(estimator)
        model.set_params(**params)
        # Обучаем модель
        model.fit(X_train, y_train)
        return model

class ModelSelection:
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict
        self.best_results_list = []

    
    def choose(self, X_train, y_train, X_valid, y_valid):
        best_model_name = None
        best_valid_score = -1  # Инициализируем с отрицательного значения
        for idx, grid in enumerate(self.grids):
            model_name = self.grid_dict[idx]
            print(f"Estimator: {model_name}")

            pbar = tqdm(total=1, desc="Fitting model")
        
            # Обучаем модель
            grid.fit(X_train, y_train)
            pbar.update()
            # Закрываем прогресс-бар
            pbar.close()

            # Получаем лучший набор параметров и оценки
            best_params = grid.best_params_
            best_model = grid.best_estimator_
            train_score = best_model.score(X_train, y_train)
            valid_score = best_model.score(X_valid, y_valid)

             # Печать результатов для текущей модели
            print(f"Best params: {best_params}")
            print(f"Best training accuracy: {train_score:.3f}")
            print(f"Validation set accuracy score for best params: {valid_score:.3f}")
            print("-" * 50)

            # Запись результатов
            self.best_results_list.append({
                'model': model_name,
                'params': best_params,
                'train_score': train_score,
                'valid_score': valid_score
            })

            # Сравниваем с текущим лучшим значением
            if valid_score > best_valid_score:
                best_valid_score = valid_score
                best_model_name = model_name
                
        # Печать лучшей модели в конце
        print(f"\nClassifier with best validation set accuracy:: {best_model_name}")

        return best_model_name

    def best_results(self):
        # Создаем DataFrame из списка лучших результатов
        return pd.DataFrame(self.best_results_list)

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [21]:
class Finalize:
    def __init__(self, estimator):
        self.estimator = estimator

    def final_score(self, X_train, y_train, X_test, y_test):
        # Обучаем модель на обучающем наборе
        self.estimator.fit(X_train, y_train)
        # Получаем прогнозы и считаем точность
        accuracy = self.estimator.score(X_test, y_test)
        print(f"Accuracy of the final model is {accuracy:.6f}")
        return accuracy

    def save_model(self, path):
        joblib.dump(self.estimator, path)
        print(f"Model successfully saved to {path}")

## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [25]:
df = pd.read_csv("data/checker_submits.csv")

In [27]:
preprocessing = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('onehot_encoder', MyOneHotEncoder('dayofweek'))
])

In [29]:
data = preprocessing.fit_transform(df)

In [31]:
X_train, X_valid, X_test, y_train, y_valid, y_test = TrainValidationTest().split(data.drop('dayofweek', axis=1), data['dayofweek'])

In [33]:
# Set up GridSearchCV instances for multiple models
svm_params = [{'kernel': ('linear', 'rbf', 'sigmoid'), 'C': [0.01, 0.1, 1, 1.5, 5, 10], "random_state":[21]}]
gs_svm = GridSearchCV(estimator=SVC(), param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=1)

dt_params = [{'max_depth': [None, 10, 20, 30], "random_state":[21]}]
gs_tree = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=dt_params, scoring='accuracy', cv=2, n_jobs=1)

rf_params = [{'n_estimators': [10, 50, 100], 'max_depth': [None, 10, 20], "random_state":[21]}]
gs_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=1)

grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}

In [35]:
model_selection = ModelSelection(grids, grid_dict)
best_model_name = model_selection.choose(X_train, y_train, X_valid, y_valid)
best_results_df = model_selection.best_results()

# Create an instance of Finalize with the best model found
model_key = next(key for key, value in grid_dict.items() if value == best_model_name)
best_estimator = grids[model_key]  # Get the best estimator from the grids
finalizer = Finalize(best_estimator.best_estimator_)

# Get final score and save the model
accuracy = finalizer.final_score(X_train, y_train, X_test, y_test)

Estimator: SVM


Fitting model:   0%|          | 0/1 [00:00<?, ?it/s]

Best params: {'C': 10, 'kernel': 'linear', 'random_state': 21}
Best training accuracy: 0.762
Validation set accuracy score for best params: 0.678
--------------------------------------------------
Estimator: Decision Tree


Fitting model:   0%|          | 0/1 [00:00<?, ?it/s]

Best params: {'max_depth': None, 'random_state': 21}
Best training accuracy: 1.000
Validation set accuracy score for best params: 0.870
--------------------------------------------------
Estimator: Random Forest


Fitting model:   0%|          | 0/1 [00:00<?, ?it/s]

Best params: {'max_depth': 20, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.999
Validation set accuracy score for best params: 0.900
--------------------------------------------------

Classifier with best validation set accuracy:: Random Forest
Accuracy of the final model is 0.890533


In [35]:
model_filename = f'data/{best_model_name}_{accuracy:.2f}.joblib'
finalizer.save_model(model_filename)

Model successfully saved to data/Random Forest_0.89.joblib
